In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/deepfdata/SampleSubmission.csv
/kaggle/input/deepfdata/Train.csv
/kaggle/input/deepfdata/Test.csv
/kaggle/input/how-to-get-32gb-ram/train_labels.parquet
/kaggle/input/how-to-get-32gb-ram/__results__.html
/kaggle/input/how-to-get-32gb-ram/train.parquet
/kaggle/input/how-to-get-32gb-ram/submission.csv
/kaggle/input/how-to-get-32gb-ram/__notebook__.ipynb
/kaggle/input/how-to-get-32gb-ram/__output__.json
/kaggle/input/how-to-get-32gb-ram/custom.css
/kaggle/input/submission-1-2/submission1 (2).csv


In [77]:
sample = pd.read_csv('/kaggle/input/deepfdata/SampleSubmission.csv') 
test = pd.read_csv('/kaggle/input/deepfdata/Test.csv')
train = pd.read_csv('/kaggle/input/deepfdata/Train.csv')

In [64]:
train 

,building,File,Cooling Setpoint,EUI,Cooling COP,Operating Hours,WWR,Equipment Heat Gain,Internal Wall Rt,Internal Floor Rt,...,Wall Rt,Start Time,windows g-value,Roof Rt,Boiler Efficiency,Internal Mass,Permeability,Total Floors Area,random load mesures,Operational Energy
0,Building_0,Shape0_10851,26.804565,37.155511,4.430542,12.166667,"(0.771636962890625,0.436004638671875,0.5746520...",15.463379,2.511920,1.919918,...,4.640045,8.778931,0.381354,2.912090,0.921890,44.441528,3.942261,18210.25,"{'Cooling': '6743.0007375945:C', 'Lights': '10...",135322.229618
1,Building_10,Shape0_11866,25.219604,64.131327,2.855347,9.166667,"(0.451275634765625,0.061151123046875,0.2780212...",11.646973,2.080851,2.822443,...,3.621094,8.131470,0.556696,4.019824,0.908726,41.702271,5.122925,13110.25,"{'Cooling': '6485.8495970786:C', 'Lights': '31...",168155.546796
2,Building_10001,Shape0_15816,26.691040,31.992473,2.863892,10.166667,"(0.324163818359375,0.669464111328125,0.6951721...",9.899902,2.274071,2.341140,...,4.595792,7.406128,0.572552,5.745949,0.911157,24.244995,4.495239,13561.60,"{'Cooling': '3861.976104307:C', 'Lights': '486...",108467.281862
3,Building_10002,Shape0_15566,25.468384,40.932114,3.922485,10.333333,"(0.772406005859375,0.924346923828125,0.5703674...",12.196777,2.541613,2.390324,...,5.310859,7.831909,0.520404,4.394614,0.929751,24.655151,6.311646,21678.00,"{'Cooling': '10788.0922449597:C', 'Lights': '7...",177465.271945
4,Building_10003,Shape0_3155,25.152832,57.792356,2.828613,10.833333,"(0.3538818359375,0.2387451171875,0.07746582031...",12.552734,1.586183,2.228146,...,4.190710,8.794434,0.475854,5.271557,0.947480,29.201660,7.530762,14200.16,"{'Cooling': '4314.2210067885:C', 'Lights': '59...",205165.175932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54595,Building_9992,Shape0_14205,25.408325,63.043619,3.872192,10.333333,"(0.889080810546875,0.498736572265625,0.2994445...",9.901855,2.779069,1.589646,...,5.896495,8.347046,0.409735,5.934082,0.924028,23.314819,6.210571,11026.24,"{'Cooling': '4817.8812345313:C', 'Lights': '86...",173783.517217
54596,Building_9993,Shape0_20612,25.150330,52.031822,2.593689,13.000000,"(0.735739135742188,0.707504272460938,0.3488006...",10.283936,1.890247,1.951719,...,4.930336,8.463562,0.529129,5.935157,0.926541,32.026062,4.032898,13899.20,"{'Cooling': '8970.7636194456:C', 'Lights': '86...",180800.175570
54597,Building_9994,Shape0_12286,26.598511,64.868078,3.460815,14.333333,"(0.435455322265625,0.242205810546875,0.3606384...",8.287598,2.565733,2.496305,...,3.282183,8.393188,0.476422,4.655868,0.969507,18.088989,7.244019,13344.80,"{'Cooling': '2182.6635332645:C', 'Lights': '65...",216412.881062
54598,Building_9995,Shape0_10572,25.040894,45.606632,4.498901,9.333333,"(0.902593994140625,0.523016357421875,0.3628356...",9.830566,1.811888,1.637205,...,4.140929,8.894165,0.524615,5.439936,0.926499,39.080200,4.815308,21601.00,"{'Cooling': '10687.6493568836:C', 'Lights': '1...",197029.771099


In [75]:
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mean
from sklearn.linear_model import LinearRegression
plt.style.use("ggplot")
import catboost as cb 

# import lightgbm as lgb 
# model = ltb.LGBMRegressor()

In [89]:

def prepross(data):
    wwr = []
    
    for x in data['WWR']: 
        test_map = x.replace('(' , '').replace(')' , '')
        tupe = tuple(map(float, test_map.split(',')))
        wwr.append(mean(tupe))
    data[['Cooling', 'Lights']] = data['random load mesures'].str.extract(r'Cooling\': \'(?P<Cooling>[0-9\.]+):C\', \'Lights\': \'(?P<Lights>[0-9\.]+):C')
    data['Cooling']=data['Cooling'].astype(float)
    data['Lights']=data['Lights'].astype(float)
    data.drop(columns=['random load mesures','building','File'], inplace= True)
    data['WWR'] = wwr
    return data 
    
def feature_engineer(data): 
    data['energy_unit'] = (data['EUI'] * data['Total Floors Area'] ) / data['Number of Floors']
    data = data.drop(['EUI', 'Total Floors Area','Number of Floors'], axis = 1 )
    return data

In [78]:

train = prepross(train)
train = feature_engineer(train)
test = prepross(test)
test = feature_engineer(test)

,Cooling Setpoint,Cooling COP,Operating Hours,WWR,Equipment Heat Gain,Internal Wall Rt,Internal Floor Rt,Infiltration,Ground Floor Rt,Occupancy,...,Start Time,windows g-value,Roof Rt,Boiler Efficiency,Internal Mass,Permeability,Operational Energy,Cooling,Lights,energy_unit
0,26.804565,4.430542,12.166667,0.493903,15.463379,2.511920,1.919918,0.210,3.445636,23.231812,...,8.778931,0.381354,2.912090,0.921890,44.441528,3.942261,135322.229618,6743.000738,10475.824219,135322.229618
1,25.219604,2.855347,9.166667,0.405573,11.646973,2.080851,2.822443,0.222,3.484326,19.027710,...,8.131470,0.556696,4.019824,0.908726,41.702271,5.122925,168155.546796,6485.849597,3108.639378,168155.546796
2,26.691040,2.863892,10.166667,0.497308,9.899902,2.274071,2.341140,0.221,5.081571,20.664673,...,7.406128,0.572552,5.745949,0.911157,24.244995,4.495239,108467.281862,3861.976104,4866.355447,108467.281862
3,25.468384,3.922485,10.333333,0.758344,12.196777,2.541613,2.390324,0.245,3.329946,19.004517,...,7.831909,0.520404,4.394614,0.929751,24.655151,6.311646,177465.271945,10788.092245,7679.311052,177465.271945
4,25.152832,2.828613,10.833333,0.279944,12.552734,1.586183,2.228146,0.313,3.493092,18.679199,...,8.794434,0.475854,5.271557,0.947480,29.201660,7.530762,205165.175932,4314.221007,5904.960314,205165.175932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54595,25.408325,3.872192,10.333333,0.435126,9.901855,2.779069,1.589646,0.262,6.199955,24.426880,...,8.347046,0.409735,5.934082,0.924028,23.314819,6.210571,173783.517217,4817.881235,8678.648007,173783.517217
54596,25.150330,2.593689,13.000000,0.493518,10.283936,1.890247,1.951719,0.214,4.237862,15.292664,...,8.463562,0.529129,5.935157,0.926541,32.026062,4.032898,180800.175570,8970.763619,8686.377313,180800.175570
54597,26.598511,3.460815,14.333333,0.333502,8.287598,2.565733,2.496305,0.298,3.153802,22.640991,...,8.393188,0.476422,4.655868,0.969507,18.088989,7.244019,216412.881062,2182.663533,6513.532089,216412.881062
54598,25.040894,4.498901,9.333333,0.538287,9.830566,1.811888,1.637205,0.214,3.045919,24.218140,...,8.894165,0.524615,5.439936,0.926499,39.080200,4.815308,197029.771099,10687.649357,10015.911978,197029.771099


In [91]:
X = np.array(train['energy_unit'])
X = X.reshape(-1,1)
y = np.array(train['Operational Energy'])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# model = cb.CatBoostRegressor(loss_function='RMSE')
reg = LinearRegression().fit(X, y)



In [94]:
from sklearn.metrics import mean_squared_error as MSE
pred = reg.predict(np.array(X_test).reshape(-1,1))
  
# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))



RMSE :  0.000000


In [96]:
result = reg.predict(np.array(test['energy_unit']).reshape(-1,1)) 

In [97]:

sample_clean = pd.read_csv('/kaggle/input/submission-1-2/submission1 (2).csv')
sample_clean['prediction_label']  = result
sample_clean.to_csv('linear_regression_tutorial.csv', index=False)

In [ ]:
# X = train.drop(['building','File', 'Operational Energy', 'random load mesures'], axis = 1)
# y = train['Operational Energy']


model = cb.CatBoostRegressor(loss_function='RMSE' , depth=10, iterations= 1000, learning_rate = 0.1)
model.fit(X_train, y_train)
# optimized_cat = grid.best_estimator_

In [ ]:
from sklearn.metrics import mean_squared_error as MSE
pred = model.predict(X_test)
  
# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))


In [ ]:
print(grid.best_params_)
print(grid.best_estimator_)
print(gird.best_score_)

In [ ]:
test[['Cooling', 'Lights']] = test['random load mesures'].str.extract(r'Cooling\': \'(?P<Cooling>[0-9\.]+):C\', \'Lights\': \'(?P<Lights>[0-9\.]+):C')
wwr = []
from statistics import mean
for x in test['WWR']:
    test_map = x.replace('(' , '').replace(')' , '')
    tupe = tuple(map(float, test_map.split(',')))
    wwr.append(mean(tupe)) 
test['WWR'] = wwr
test['Cooling'] = test['Cooling'].astype('float32')
test['Lights'] = test['Lights'].astype('float32')


In [ ]:
# test['Cooling'] = test['Cooling'].astype('float32')
# test['Lights'] = test['Lights'].astype('float32')
# test =  test.drop([ 'building','File' , 'random load mesures'], axis =1 )
result = model.predict(test)

In [ ]:
result.shape

In [ ]:
test_again = pd.read_csv("/kaggle/input/deepfdata/Test.csv")
# test_again['ID'] = test_again.apply(lambda x: f"Building_{x['building'].split('')[1]}_Town{x['Town']}", axis=1)
test_again.head() 
# test_again['ID'] = test_again.apply(lambda x : f"Building_{x['building'].split('')[1]}_Town{x['Town']}", axis = 1 )

In [ ]:
test_again = pd.read_csv("/kaggle/input/deepfdata/Test.csv")
test_again.drop(columns={'File', 'Cooling Setpoint', 'EUI', 'Cooling COP',
       'Operating Hours', 'WWR', 'Equipment Heat Gain', 'Internal Wall Rt',
       'Internal Floor Rt', 'Infiltration', 'Ground Floor Rt',
       'Number of Floors', 'Occupancy', 'Light Heat Gain', 'Windows Rt',
       'Height', 'Heating COP', 'Heating Setpoint', 'Town', 'Wall Rt',
       'Start Time', 'windows g-value', 'Roof Rt', 'Boiler Efficiency',
       'Internal Mass', 'Permeability', 'Total Floors Area',
       'random load mesures'}, inplace= True)

In [ ]:
test_again['Operational Energy'] = result

In [ ]:
test_again.to_csv('submission_1_xgb_simple.csv', index = False )

In [ ]:
test_again.rename(columns = {'building':'submission_id'}, inplace = True)

In [ ]:
test_again.to_csv('submission_1_xgb_simple.csv', index = False )

In [ ]:
sample_clean = pd.read_csv('/kaggle/input/submission-1-2/submission1 (2).csv')

In [ ]:
sample_clean['prediction_label']  = result

In [ ]:
sample_clean.to_csv('catboost_800mse_iterations.csv', index=False)

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train) 
lgb_eval = lgb.Dataset(X_test, y_test, reference = lgb_train)

In [ ]:
params = {
    'task' : 'train',
    'boosting' : 'gbdt', 
    'objective' : 'regression', 
    'learning_rate' : 0.05 , 
    'metric' : {'l2','l1'}, 
    'verbose' : -1 
}

model = lgb.train(params, 
                 train_set = lgb_train,
                 valid_sets = lgb_eval , 
                 early_stopping_rounds = 40 )


In [ ]:
y_pred = model.predict(X_test) 
mse = MSE(y_test, y_pred) 
rmse = mse**(0.5)
print("RMSE", rmse)